# 4-4 AutoGraph的机制原理
Autograph机制可以将动态图转换成静态计算图，兼收执行效率和编码效率之利。

当然Autograph机制能够转换的代码并不是没有任何约束的，有一些编码规范需要遵循，否则可能会转换失败或者不符合预期。


In [1]:
import tensorflow as tf
import numpy as np

## 一、Autograph的机制原理

In [11]:
@tf.function(autograph = True)
def myadd(a, b):
    for i in tf.range(3):
        tf.print(i)
    c = a + b
    print("tracing")
    # 改成tf.print的执行顺序就会正常
    return c

In [12]:
myadd(tf.constant("hello"), tf.constant("world"))

tracing
0
1
2


<tf.Tensor: shape=(), dtype=string, numpy=b'helloworld'>

执行被@tf.function修饰的函数一共发生了两件事情

1. 第一件事情就是创建计算图。即创建一个静态的计算图，跟踪执行一遍函数体中的Python代码，确定各个变量的Tensor类型，并根据执行
顺序将算子添加到计算图中。在这个过程中，如果开启了autograph = True(默认), 将会把Python控制流转为TensorFlow图内控制流。主要就是
将if语句转为tf.cond, 将while和for转为tf.while_loop算子, 并在必要的时候添加tf.control_dependencies指定执行顺序依赖关系\
2. 第二件事情就是执行计算图

In [13]:
myadd(tf.constant("good"), tf.constant("morning"))  # 传入相同类型的参数, 再次调用就不会执行tracing

0
1
2


<tf.Tensor: shape=(), dtype=string, numpy=b'goodmorning'>

**当我们传入不同类型的参数时, 再看看会发生什么？**

In [14]:
myadd(tf.constant(2), tf.constant(1))

tracing
0
1
2


<tf.Tensor: shape=(), dtype=int32, numpy=3>

**这里需要注意的就是如果传入的不是Tensor类型, 那么每次都会重新创建计算图**

In [15]:
myadd("sss", "bbb")
myadd("ggg", "")

tracing
0
1
2
tracing
0
1
2


<tf.Tensor: shape=(), dtype=string, numpy=b'ggg'>

**下面做一些小实验**

In [16]:
@tf.function()
def myadd1(a, b):
    for i in range(3):
        print(i)
    c = a + b
    print("tracing")
    return c

In [22]:
myadd(tf.constant(1.0), tf.constant(2.0))

tracing
0
1
2


<tf.Tensor: shape=(), dtype=float32, numpy=3.0>

In [23]:
myadd(tf.constant(1), tf.constant(2))

0
1
2


<tf.Tensor: shape=(), dtype=int32, numpy=3>

In [25]:
myadd(tf.constant(1.0), tf.constant(2.))

0
1
2


<tf.Tensor: shape=(), dtype=float32, numpy=3.0>

## 二，重新理解Autograph的编码规范


了解了以上Autograph的机制原理，我们也就能够理解Autograph编码规范的3条建议了。

1，被@tf.function修饰的函数应尽量使用TensorFlow中的函数而不是Python中的其他函数。例如使用tf.print而不是print.

解释：Python中的函数仅仅会在跟踪执行函数以创建静态图的阶段使用，普通Python函数是无法嵌入到静态计算图中的，所以
在计算图构建好之后再次调用的时候，这些Python函数并没有被计算，而TensorFlow中的函数则可以嵌入到计算图中。使用普通的Python函数会导致
被@tf.function修饰前【eager执行】和被@tf.function修饰后【静态图执行】的输出不一致。

2，避免在@tf.function修饰的函数内部定义tf.Variable. 

解释：如果函数内部定义了tf.Variable,那么在【eager执行】时，这种创建tf.Variable的行为在每次函数调用时候都会发生。但是在【静态图执行】时，这种创建tf.Variable的行为只会发生在第一步跟踪Python代码逻辑创建计算图时，这会导致被@tf.function修饰前【eager执行】和被@tf.function修饰后【静态图执行】的输出不一致。实际上，TensorFlow在这种情况下一般会报错。

**修正解释**：如果在Autograph中定义Variable的话，由于创建静态图的时候不知道会调用几次函数进行跟踪，而Variable在静态图中是一个边（不像在Eager模式下面会受到作用域的限制）, 所以可能会多次创建相同的Variable张量

3，被@tf.function修饰的函数不可修改该函数外部的Python列表或字典等数据结构变量。

解释：静态计算图是被编译成C++代码在TensorFlow内核中执行的。Python中的列表和字典等数据结构变量是无法嵌入到计算图中，它们仅仅能够在创建计算图时被读取，在执行计算图时是无法修改Python中的列表或字典这样的数据结构变量的。